In [19]:
import numpy as np

%run -i runup_funcs.py

print(calc_L0(10))

print(func_list)

for item in func_list:
    cmd = "print({}())".format(item, item)
    print(item)
    exec(cmd)

156.13099917314932
['A17', 'D20', 'H86', 'NH91', 'P14', 'P16', 'R01', 'S06', 'S11', 'V12']
A17
0.3498845445000393
D20
5.574916526297034
H86
0.6933375474091239
NH91
1.764186045187383
P14
0.7407072495843278
P16
0.9333809511662428
R01
0.6747429110409386
S06
0.7371541008103424
S11
1.4210386883731967
V12
0.2105119278636574
